In [1]:
#!/usr/bin/env python
import os
import sys
import rospy
import copy
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import moveit_msgs.msg
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from apriltags_ros.msg import AprilTagDetection, AprilTagDetectionArray
from math import pi, cos, sin
import math
from numpy.linalg import inv
import scipy.io
from mat4py import loadmat
from docx import Document
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from dig_grasp.msg import stone_pose
from sensor_msgs.msg import Image, CameraInfo
import message_filters

from visualization_msgs.msg import Marker
from visualization_msgs.msg import MarkerArray
from geometry_msgs.msg import Point
import moveit_commander

import urx
import math3d as m3d
import logging

import matplotlib.pyplot as plt

rospy.init_node('pokegrasp', anonymous=True) #initialize the node
scene = moveit_commander.PlanningSceneInterface()

In [2]:

logging.basicConfig(level=logging.WARN)
rob = urx.Robot("192.168.1.102")


In [3]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()


True

In [4]:

def go_to_home():
    
    
    #pick 2 
    #home_position = [123.48,-88.41,-107.01,-76.67,89.75,-56.2]
    home_position = [123.42,-86.47,-92.93,-92.68,89.77,-56.13]
    Hong_joint0 = math.radians(home_position[0])
    Hong_joint1 = math.radians(home_position[1])
    Hong_joint2 = math.radians(home_position[2])
    Hong_joint3 = math.radians(home_position[3])
    Hong_joint4 = math.radians(home_position[4])
    Hong_joint5 = math.radians(home_position[5])
    
    # picking only
    '''
    Hong_joint0 = math.radians(106.83)
    Hong_joint1 = math.radians(-77.46)
    Hong_joint2 = math.radians(-125.81)
    Hong_joint3 = math.radians(-68.64)
    Hong_joint4 = math.radians(89.58)
    Hong_joint5 = math.radians(-73.07)
    '''

    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.3, 0.4)
    
def go_to_pre_home():
    
    
    #home_position = [142.55, -93.79,-110.91,-66.98,90.43,-37.36]
    #pick and place
    #home_position = [139.25,-99.86,-103.95,-67.93,90.22,-40.67]
    
    #pick 2 
    home_position = [123.48,-88.41,-107.01,-76.67,89.75,-56.2]
    Hong_joint0 = math.radians(home_position[0])
    Hong_joint1 = math.radians(home_position[1])
    Hong_joint2 = math.radians(home_position[2])
    Hong_joint3 = math.radians(home_position[3])
    Hong_joint4 = math.radians(home_position[4])
    Hong_joint5 = math.radians(home_position[5])
    
    # picking only
    '''
    Hong_joint0 = math.radians(106.83)
    Hong_joint1 = math.radians(-77.46)
    Hong_joint2 = math.radians(-125.81)
    Hong_joint3 = math.radians(-68.64)
    Hong_joint4 = math.radians(89.58)
    Hong_joint5 = math.radians(-73.07)
    '''

    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.7, 0.7)

    
def rotate_vector(v_i,angle):
    R = np.array([[math.cos(angle),-math.sin(angle)],[math.sin(angle),math.cos(angle)]])
    v_f = R

In [74]:
def execute():
    global pose_x
    global pose_y
    global pose_z
    global yaw
    global pitch
    global normal
    
    
    global tcpTstone
    
    global cal
    
    global offset_d, psi
    global dig_depth 
    global g_close
    
    ####################### change tcp pose ############################
    rob.set_tcp((0.013, 0.0, 0.32601, 0, 0, 0)) 
    ####################################################################
    #rob.translate_tool((0, 0, -0.05-0.045), acc=0.06, vel=0.06)
    #rob.translate_tool((0, 0, -0.05-0.045-0.05), acc=0.1, vel=0.3)
    
    camPstone = np.array([pose_x, pose_y, pose_z, 1])
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173 + 0.05, -0.0934057 -0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    print eeTcam

    yaxis = (0, 1, 0)
    zaxis = (0, 0, 1)
    Ry = tf.transformations.rotation_matrix(pi/2, yaxis)
    Rz = tf.transformations.rotation_matrix(-pi/2, zaxis)
    eeTtcp = np.matmul(Ry, Rz)
    ####################### change tcp pose ############################
    eeTtcp[:3,3] = np.array([0.32601,-0.011,0])
    #eeTtcp[:3,3] = np.array([0.28118,-0.013/2,0])
    #eeTtcp[:3,3] = np.array([0.36386,-0.024/2,0])
    ####################################################################
    tcpTee = inv(eeTtcp)
    print tcpTee
    
    eeTstone = np.matmul(eeTcam.get_matrix(), camPstone)
    print eeTstone
    tcpTstone = np.matmul(tcpTee, np.transpose(eeTstone))
    print "tcpTstone", tcpTstone
    
    n_normalize = normal / np.linalg.norm(normal)
    #if n_normalize[2] > 0:
    #    n_normalize[0] = -n_normalize[0]
    #    n_normalize[1] = -n_normalize[1]
    #    n_normalize[2] = -n_normalize[2]
    print "n_normalize", n_normalize
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    eeTnormal = np.matmul(eeTcam.get_matrix(), n_normalize)
    tcpTnormal = np.matmul(tcpTee, np.transpose(eeTnormal))
    
    
    yaw_difference = (math.atan2(tcpTnormal[1],tcpTnormal[0]) - yaw)%(2*pi)
    
    if  yaw_difference > pi/2.0 and yaw_difference < 3.0 * pi/2.0:
        yaw = yaw + pi
        print(' yaw opposite',yaw)
        
    
    if  yaw > pi:
        yaw = yaw - 2*pi
        
    #tcpTy=???[0,1]
    #alpha = already from dipn
    #if tcpTnormal[1] > 0:
    R = np.array([[math.cos(yaw),math.sin(yaw)],[-math.sin(yaw),math.cos(yaw)]])
    tcpTnormal_after_yaw_xy = np.matmul(R, tcpTnormal[:2])
    if tcpTnormal_after_yaw_xy[1] > 0:
        gamma = pi-math.atan2(tcpTnormal_after_yaw_xy[1], tcpTnormal[2])
        print "gamma case 1"
       # print math.atan2(tcpTnormal[1], tcpTnormal[2])
    elif tcpTnormal_after_yaw_xy[1] < 0:
        gamma = -(math.atan2(tcpTnormal_after_yaw_xy[1], tcpTnormal[2])+pi)
        print "gamma case 2"
        #print math.atan2(tcpTnormal[1], tcpTnormal[2])
        
    beta = -math.asin(tcpTnormal_after_yaw_xy[0]/ np.linalg.norm(tcpTnormal[:3])) #tune psi
    #if tcpTnormal[0] > 0:
    #    #beta = -(pi/2+math.atan2(tcpTnormal[2], tcpTnormal[0])) #tune psi
    #    beta = -pi+math.atan2(tcpTnormal[2], tcpTnormal[0]) #tune psi
    #    print "beta case 1"
    #    print math.atan2(tcpTnormal[2], tcpTnormal[0])
    #elif tcpTnormal[0] < 0:
        #beta = -(pi/2+math.atan2(tcpTnormal[2], -tcpTnormal[0])) #tune psi
        #beta = -pi+math.atan2(tcpTnormal[2], -tcpTnormal[0]) #tune psi
        #print "beta case 2"
        #print math.atan2(tcpTnormal[2], tcpTnormal[0])
    
    
    
    print "gamma", gamma*180/pi, "beta", beta*180/pi, 'yaw', yaw*180/pi
    
    
    
    print "tcpTnormal", tcpTnormal
    print "tcpTnormal_after_yaw_xy", tcpTnormal_after_yaw_xy
    
    #raw_input()
    
    #print "alpha", alpha*180/pi, "beta", beta*180/pi
    
    #move = m3d.Transform((tcpTstone[0,0]+0.004  + 0.002, tcpTstone[1,0]+0.015, 0, 0, 0, 0))
    move = m3d.Transform((tcpTstone[0,0]+0.03 + offset[0], tcpTstone[1,0]+0.025 + offset[1], 0, 0, 0, 0))
    # direct grasp
    #move = m3d.Transform((tcpTstone[0,0]+0.004  + 0.002, tcpTstone[1,0]+0.012, 0, 0, 0, 0))
    rob.add_pose_tool( move, acc=0.15, vel=0.2, wait=True, command="movel", threshold=None)
    ## extenable finger
    #rob.translate_tool((-0.002*cos(pitch), 0.006, tcpTstone[2,0]+0.055+0.045  - 0.005), acc=0.02, vel=0.03)
    ##new fixed finger
    rob.translate_tool((-0.002*cos(pitch) - 0.002, 0.006 - 0.006- 0.001, tcpTstone[2,0] -0.05+ offset[2]), acc=0.03, vel=0.1)
    print('tcpTstone[2,0]',tcpTstone[2,0])
    #z+0.23
    #raw_input()
    
    
    #rotation = m3d.Transform((0,0,0,0,beta,alpha))
    ### disable rotation: rob.add_pose_tool( rotation, acc=0.1, vel=0.1, wait=True, command="movel", threshold=None)
    #rotation = m3d.Transform((0,0,0,0,beta,0))
    #rob.add_pose_tool( rotation, acc=0.1, vel=0.1, wait=True, command="movel", threshold=None)
    #
    
    
    #rob.translate_tool((0,0, 0.008+0.02), acc=0.02, vel=0.03)
    
    
    raw_input()
    
    beta = beta #+ 20.0*pi/180.0
    
    
    
    ## rotate z-axis
    rotation = m3d.Transform((0,0,0,0,0,yaw))
    rob.add_pose_tool( rotation, acc=0.1, vel=0.5, wait=True, command="movel", threshold=None)
    rotation = m3d.Transform((0,0,0,gamma,0,0))
    rob.add_pose_tool( rotation, acc=0.1, vel=0.2, wait=True, command="movel", threshold=None)
    rotation = m3d.Transform((0,0,0,0,beta,0))
    rob.add_pose_tool( rotation, acc=0.1, vel=0.2, wait=True, command="movel", threshold=None)
    
    
    raw_input('press enter to continue')
    
    
    ###########################Choose d and psi######################################
    rob.translate_tool((offset_d , 0, 0), acc=0.1, vel=0.1) # d for domino
    rotation = m3d.Transform((0,0,0,0,psi,0))
    rob.add_pose_tool( rotation, acc=0.1, vel=0.1, wait=True, command="movel", threshold=None)
    #################################################################################
    
    #rob.translate_tool((0, -0.005, 0), acc=0.02, vel=0.03)
    
    print 0.1226-(tcpTstone[2,0]+0.2)
    
    rob.translate_tool((0, 0, dig_depth ), acc=0.03, vel=0.05) #1.1, 90deg ## new fixed finger
    
    
    
    
    Robotiq.goto(robotiq_client, pos=g_close, speed=0.003, force=10, block=False) 
    rospy.sleep(1)
    #raw_input()
    #rotation = m3d.Transform((0,0,0,0,-pitch,0))
    #rob.add_pose_tool( rotation, acc=0.02, vel=0.03, wait=True, command="movel", threshold=None)
    
    #########################ONLY PICKING################################
    # return home
    #raw_input()
 #   rob.translate_tool((0, 0, -0.08), acc=0.02, vel=0.03)
 #   rob.movel((0, -0.12, 0, 0, 0, 0), acc=0.2, vel=0.2, wait=True, relative=True)
 #   go_to_home()
 #   Robotiq.goto(robotiq_client, pos=0.024, speed=0.2, force=10, block=False)
    
    
    ##########################For PLACING################################
    rob.translate_tool((0, 0, -0.08), acc=0.2, vel=0.2)
    joint = rob.getj()
    '''if joint[5] > pi:
        rob.movej((joint[0],joint[1],joint[2],joint[3],joint[4],joint[5]-pi),0.5,0.8)
    elif joint[5] < -pi:
        rob.movej((joint[0],joint[1],joint[2],joint[3],joint[4],joint[5]+pi),0.5,0.8)'''
    '''
    ## Hin
    rob.movel((0, 0.16, 0, 0, 0, 0), acc=0.2, vel=0.2, wait=True, relative=True)
    
    Robotiq.goto(robotiq_client, pos=0.022, speed=0.5, force=10, block=False)
    rospy.sleep(1)
    
    rob.movel((0, -0.2-0.16, 0, 0, 0, 0), acc=0.2, vel=0.2, wait=True, relative=True)
    '''
    
    #rob.movel((0, -0.2, 0, 0, 0, 0), acc=0.2, vel=0.2, wait=True, relative=True)
    '''
    rob.movel((-0.35, -0.25, 0, 0, 0, 0), acc=0.2, vel=0.2, wait=True, relative=True)
    rotation = m3d.Transform((0,0,0,0,0,-alpha+pi))
    rob.add_pose_tool( rotation, acc=0.1, vel=0.1, wait=True, command="movel", threshold=None)
    rob.movel((0, 0, -0.173, 0, 0, 0), acc=0.1, vel=0.1, wait=True, relative=True)
    #go_to_home()
    #Robotiq.goto(robotiq_client, pos=0.008+0.006+0.008, speed=0.2, force=10, block=False)
    '''

In [75]:
def stone_pose_callback(msg):
    print(msg)
    
    global pose_x
    global pose_y
    global pose_z
    global yaw
    global pitch
    global normal
    
    pose_x = msg.x
    pose_y = msg.y
    pose_z = msg.z
    yaw = msg.yaw
    pitch = msg.pitch
    normal = msg.normal
    
    print 'pose_x = ', pose_x, ',pose_y = ', pose_y, ',pose_z = ', pose_z
    
    
pose_topic = '/stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(pose_topic, stone_pose, stone_pose_callback)

In [76]:
global is_send_stone_img
is_send_stone_img = 0
stone_img_count = 0

pose_x = 0
pose_y = 0
yaw = 0
pitch = 0
bridge = CvBridge()

def image_callback(color, a_depth):
    global is_send_stone_img
    global stone_img_count
    
    #print 'enter img callback', is_send_img
    
    
    if is_send_stone_img == 1:
        print "dsag"
        #print(color)
        #print(a_depth)
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        rospy.sleep(0.5)
        cv2_img = bridge.imgmsg_to_cv2(color, "bgr8")
        cv2.imwrite('../Mask_RCNN/samples/stones/JPEGImages/'+str(stone_img_count)+'.jpeg', cv2_img)
        cv2_depth_img = bridge.imgmsg_to_cv2(a_depth, desired_encoding="passthrough")
        depth_array = np.array(cv2_depth_img, dtype=np.float32)
        print depth_array
        #print depth_array[36,532]
        cv2.imwrite('../Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.jpeg', cv2_depth_img)
        np.save('../Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.npy', depth_array)
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        stone_img_count = stone_img_count + 1
        is_send_stone_img = 0
        '''
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        #cv2_img = bridge.imgmsg_to_cv2(msg, "bgr8")
        #cv2.imwrite('/home/yuhin/ws_moveit/src/pickpack/Mask_RCNN/log/'+str(carton_img_count)+'.jpeg', cv2_img)
        rospy.sleep(1)
        #print('from image callback, publish /carton_img_index')
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        #stone_img_count = stone_img_count + 1
        is_send_stone_img = 0
        '''

In [77]:
#image_topic = '/camera/color/image_raw' #"/usb_cam/image_raw"

# Set up your subscriber and define its callback
#rospy.Subscriber(image_topic, Image, image_callback)
image_color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
image_aligned_depth_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
#info_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/camera_info', CameraInfo)

ts = message_filters.TimeSynchronizer([image_color_sub, image_aligned_depth_sub], 10)
ts.registerCallback(image_callback)

0

In [96]:

global cal
cal = [0.005 - 0.002,0.004 - 0.007,0.03]
global offset_d, psi
offset_d = 0.008+0.001
psi = math.radians(25)

global dig_depth 
dig_depth =0.0265 + 0.0265 

global g_ap
g_ap = 0.023 

global g_close 
g_close = 0.002 


In [99]:
#stone_img_count = 24

try:
    global is_send_stone_img
    
    go_to_home()
    Robotiq.goto(robotiq_client, pos=g_ap, speed=0.01, force=10, block=False) ##domino
    go_to_home()
    
    is_send_stone_img = 1

except rospy.ROSInterruptException: pass


dsag
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
16
header: 
  seq: 47
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
x: -0.0202332080343
y: -0.0134524582232
z: 0.355
yaw: -3.01189011643
pitch: 0.0
normal: [0.8178433755771788, 0.8015908219301777, -2.767130305265206]
pose_x =  -0.0202332080343 ,pose_y =  -0.0134524582232 ,pose_z =  0.355
header: 
  seq: 47
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
x: -0.0202332080343
y: -0.0134524582232
z: 0.355
yaw: -3.01189011643
pitch: 0.0
normal: [0.8178433755771788, 0.8015908219301777, -2.767130305265206]
pose_x =  -0.0202332080343 ,pose_y =  -0.0134524582232 ,pose_z =  0.355
header: 
  seq: 47
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
x: -0.0202332080343
y: -0.0134524582232
z: 0.355
yaw: -3.01189011643
pitch: 0.0
normal: [0.8178433755771788, 0.80159082

In [98]:
execute()

<Transform:
<Orientation: 
array([[ 0.04290745, -0.01806757,  0.99891567],
       [-0.02153275,  0.9995875 ,  0.01900464],
       [-0.99884698, -0.02232484,  0.04250071]])>
<Vector: (0.12617, -0.12341, 0.00748)>
>
[[  3.74939946e-33  -1.00000000e+00  -6.12323400e-17  -1.10000000e-02]
 [  6.12323400e-17   6.12323400e-17  -1.00000000e+00  -1.92887994e-17]
 [  1.00000000e+00   0.00000000e+00   6.12323400e-17  -3.26010000e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
[[ 0.46874478 -0.1030771   0.01720676  1.        ]]
tcpTstone [[ 0.0920771 ]
 [-0.01720676]
 [ 0.14273478]
 [ 1.        ]]
n_normalize [-0.02805668  0.03364598 -0.99903992]
gamma case 1
gamma 3.01692385397 beta -3.53318362083 yaw -35.9097230792
tcpTnormal [[ 0.09715586]
 [ 0.00770562]
 [-1.19960538]
 [ 1.        ]]
tcpTnormal_after_yaw_xy [[ 0.0741712 ]
 [ 0.06322397]]
('tcpTstone[2,0]', 0.14273477623562975)


-0.220134776236
